In [1]:
import pandas as pd
import seaborn as sns
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
from datetime import datetime
import matplotlib.pyplot as plt
#from geopy.distance import geodesic
import time
from sklearn.preprocessing import LabelEncoder

In [2]:
train = pd.read_csv('./data/trainclean.csv')
test = pd.read_csv('./data/testclean.csv')

/Users/bryancombs/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (27,28,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# replace some missing values and T with -1
train = train.replace('M', -1)
train = train.replace('-', -1)
train = train.replace('T', -1)
train = train.replace(' T', -1)
train = train.replace('  T', -1)

In [4]:
test.columns

Index(['index', 'id', 'date', 'address', 'block', 'street', 'trap',
       'addressnumberandstreet', 'latitude', 'longitude', 'addressaccuracy',
       'year', 'month', 'day', 'tmax_x', 'tmin_x', 'tavg_x', 'depart_x',
       'dewpoint_x', 'wetbulb_x', 'heat_x', 'cool_x', 'sunrise_x', 'sunset_x',
       'codesum_x', 'depth_x', 'water1_x', 'snowfall_x', 'preciptotal_x',
       'stnpressure_x', 'sealevel_x', 'resultspeed_x', 'resultdir_x',
       'avgspeed_x', 'tmax_y', 'tmin_y', 'tavg_y', 'depart_y', 'dewpoint_y',
       'wetbulb_y', 'heat_y', 'cool_y', 'sunrise_y', 'sunset_y', 'codesum_y',
       'depth_y', 'water1_y', 'snowfall_y', 'preciptotal_y', 'stnpressure_y',
       'sealevel_y', 'resultspeed_y', 'resultdir_y', 'avgspeed_y',
       'species_culex_pipiens', 'species_culex_pipiens/restuans',
       'species_culex_restuans', 'species_culex_salinarius',
       'species_culex_tarsalis', 'species_culex_territans',
       'species_unspecified_culex', 'distance', 'distance.1'],
      dty

In [5]:
# test.drop[columns = ['address', 'addressanumberandstreet']]
# train.drop[columns = []]

In [6]:
lbl = LabelEncoder()
lbl.fit(list(train['trap'].values) + list(test['trap'].values))
train['trap'] = lbl.transform(train['trap'].values)
test['trap'] = lbl.transform(test['trap'].values)

In [7]:
lbl.fit(list(train['block'].values) + list(test['block'].values))
train['block'] = lbl.transform(train['block'].values)
test['block'] = lbl.transform(test['block'].values)

In [8]:
lbl.fit(list(train['street'].values) + list(test['street'].values))
train['street'] = lbl.transform(train['street'].values)
test['street'] = lbl.transform(test['street'].values)

In [9]:
train.block

0        26
1        26
2        42
3        55
4        55
5         5
6        12
7         1
8         1
9         1
10        9
11       10
12       10
13       10
14       10
15        1
16        6
17       10
18       10
19       37
20       37
21       45
22       53
23        5
24       59
25       26
26       26
27       26
28       55
29       55
         ..
10476     0
10477     0
10478     0
10479    39
10480    25
10481    25
10482    25
10483    61
10484     0
10485     0
10486     0
10487     0
10488     0
10489     0
10490    32
10491    35
10492    35
10493    58
10494    45
10495    45
10496     6
10497    60
10498     3
10499     3
10500    24
10501    35
10502    39
10503     6
10504    50
10505    27
Name: block, Length: 10506, dtype: int64

#### There is missing data in the dataset
Many missing values are entered a M, - and T in the data set.  This code sets them all to -1 do distinquish from other data

In [10]:
# replace some missing values and T with -1
test = test.replace('M', -1)
test = test.replace('-', -1)
test = test.replace('T', -1)
test = test.replace(' T', -1)
test = test.replace('  T', -1)

In [11]:
train.street.value_counts()

132    750
64     542
86     347
50     266
36     216
102    185
98     183
40     173
18     172
1      160
0      156
112    154
4      152
71     148
96     147
134    146
58     146
131    140
35     139
39     133
54     130
88     129
69     129
8      128
19     126
33     122
48     117
55     114
62     112
5      112
      ... 
51      23
16      23
10      22
114     22
119     22
75      21
27      21
70      21
128     21
79      20
61      20
76      19
14      18
92      17
7       17
45      17
87      15
11      15
47      15
133     15
101     14
135     14
124     14
137     12
91       8
83       8
109      7
73       5
110      3
67       2
Name: street, Length: 128, dtype: int64

In [12]:
train.block.value_counts()

0     1722
1      736
2      605
10     500
3      345
22     330
6      305
27     300
49     295
36     277
39     252
34     244
20     229
41     228
12     218
25     189
26     185
18     179
24     177
11     160
59     157
21     133
50     133
35     132
30     122
58     120
45     117
40     116
47     116
61     111
      ... 
54      92
28      91
17      90
5       90
43      86
60      77
53      69
23      68
51      66
44      63
32      60
42      56
37      51
9       49
7       43
33      39
57      36
63      31
52      30
48      29
8       25
64      23
46      21
62      21
31      20
14      17
56      13
19       7
38       5
16       3
Name: block, Length: 64, dtype: int64

In [13]:
#This is just a shortcut for now. If we rerun the original EDA doc, this row will kick an error out.
test['distance'] = test['distance.1']

In [14]:
test.drop(columns = 'distance.1', inplace=True)

In [15]:
#Creating interaction terms with distance and species. Input markdown here about vertical transmission.

In [16]:
train['dist_species_culex_pipiens'] = train['species_culex_pipiens'] * train['distance']
test['dist_species_culex_pipiens'] = test['species_culex_pipiens'] * test['distance']

In [17]:
train['dist_species_culex_pipiens/restuans'] = train['species_culex_pipiens/restuans'] * train['distance']
test['dist_species_culex_pipiens/restuans'] = test['species_culex_pipiens/restuans'] * test['distance']

In [18]:
train['dist_species_culex_restuans'] = train['species_culex_restuans'] * train['distance']
test['dist_species_culex_restuans'] = test['species_culex_restuans'] * test['distance']

In [19]:
train['dist_species_culex_salinarius'] = train['species_culex_salinarius'] * train['distance']
test['dist_species_culex_salinarius'] = test['species_culex_salinarius'] * test['distance']

In [20]:
train['dist_species_culex_tarsalis'] = train['species_culex_tarsalis'] * train['distance']
test['dist_species_culex_tarsalis'] = test['species_culex_tarsalis'] * test['distance']

In [21]:
train['dist_species_culex_territans'] = train['species_culex_territans'] * train['distance']
test['dist_species_culex_territans'] = test['species_culex_territans'] * test['distance']

In [22]:
test['species_unspecified_culex_dist'] = test['species_unspecified_culex'] * test['distance']

#### Month distance interaction

In visualizing the data in Tableau, we can see that the West Nile cases start to be more prolific in the mid summer with fewer cases in the early and late summer so though that distance might interact with month.

Trap data should be dummied the numbers do not mean anything on their own.  

In [23]:
train.to_csv('./data/trainw.csv')
test.to_csv('./data/testw.csv')